<a href="https://colab.research.google.com/github/avilavate/MISC/blob/master/Neural_Nets_Project_Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [10]:
import numpy as np
import os
#from scipy.misc import  imresize
from imageio import imread
import cv2
import datetime
import os

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
!ls "/content/drive/My Drive/Project_data"

train  train.csv  val  val.csv


In [12]:
!!pip install rasa==1.1.4

['Collecting rasa==1.1.4',
 '  Downloading rasa-1.1.4-py3-none-any.whl (447 kB)',
 '\x1b[?25l',
 '\x1b[K     |▊                               | 10 kB 16.2 MB/s eta 0:00:01',
 '\x1b[K     |█▌                              | 20 kB 20.7 MB/s eta 0:00:01',
 '\x1b[K     |██▏                             | 30 kB 22.8 MB/s eta 0:00:01',
 '\x1b[K     |███                             | 40 kB 23.7 MB/s eta 0:00:01',
 '\x1b[K     |███▋                            | 51 kB 26.6 MB/s eta 0:00:01',
 '\x1b[K     |████▍                           | 61 kB 28.1 MB/s eta 0:00:01',
 '\x1b[K     |█████▏                          | 71 kB 28.7 MB/s eta 0:00:01',
 '\x1b[K     |█████▉                          | 81 kB 30.6 MB/s eta 0:00:01',
 '\x1b[K     |██████▋                         | 92 kB 23.5 MB/s eta 0:00:01',
 '\x1b[K     |███████▎                        | 102 kB 24.3 MB/s eta 0:00:01',
 '\x1b[K     |████████                        | 112 kB 24.3 MB/s eta 0:00:01',
 '\x1b[K     |████████▉                     

We set the random seed so that the results don't vary drastically.

In [15]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [24]:
train_doc = np.random.permutation(open('/content/drive/MyDrive/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/MyDrive/Project_data/val.csv').readlines())
batch_size = 10

In [25]:
print(train_doc[:10])

['WIN_20180907_16_12_40_Pro_Thumbs Down_new;Thumbs Down_new;3\n'
 'WIN_20180926_17_21_29_Pro_Stop_new;Stop_new;2\n'
 'WIN_20180926_17_27_48_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n'
 'WIN_20180907_16_07_57_Pro_Left Swipe_new_Left Swipe_new;Left Swipe_new_Left Swipe_new;0\n'
 'WIN_20180907_16_25_19_Pro_Right Swipe_new;Right Swipe_new;1\n'
 'WIN_20180925_17_22_55_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180926_17_25_18_Pro_Stop_new;Stop_new;2\n'
 'WIN_20180907_16_28_20_Pro_Thumbs Up_new;Thumbs Up_new;4\n'
 'WIN_20180926_16_45_10_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n'
 'WIN_20180925_17_28_48_Pro_Stop_new;Stop_new;2\n']


In [26]:
print(val_doc[:10])

['WIN_20180926_17_09_33_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180925_17_59_48_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n'
 'WIN_20180926_17_33_14_Pro_Stop_new;Stop_new;2\n'
 'WIN_20180907_16_20_53_Pro_Right Swipe_new;Right Swipe_new;1\n'
 'WIN_20180926_17_40_21_Pro_Left_Swipe_new;Left_Swipe_new;0\n'
 'WIN_20180925_17_32_32_Pro_Stop_new;Stop_new;2\n'
 'WIN_20180907_16_21_39_Pro_Left Swipe_new_Left Swipe_new;Left Swipe_new_Left Swipe_new;0\n'
 'WIN_20180907_16_13_24_Pro_Thumbs Down_new;Thumbs Down_new;3\n'
 'WIN_20180925_17_50_24_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n'
 'WIN_20180907_16_39_59_Pro_Thumbs Up_new;Thumbs Up_new;4\n']


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [30]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = 30
    #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = 5
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image_resized=imresize(image,(self.image_height,self.image_width,3))

                    batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255
                    batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255
                    batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [33]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/MyDrive/Project_data'
val_path = '/content/drive/MyDrive/Project_data'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs =10
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
optimiser = #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)